A linear regression for weather predict based on tensorflow
===============================================
Using keras
--------------


In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import math, time, math, random
%matplotlib inline

#### Using tensonflow version: 

In [2]:
tf.__version__

'2.5.0'

## Introduce to keras

> Copyright 2015 The TensorFlow Authors. All Rights Reserved. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at 
> http://www.apache.org/licenses/LICENSE-2.0
> Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and

Keras is a deep learning API written in Python, running on top of the machine learning platform TensorFlow. It was developed with a focus on enabling fast experimentation. Being able to go from idea to result as fast as possible is key to doing good research.

Keras is:

Simple -- but not simplistic. Keras reduces developer cognitive load to free you to focus on the parts of the problem that really matter.

Flexible -- Keras adopts the principle of progressive disclosure of complexity: simple workflows should be quick and easy, while arbitrarily advanced workflows should be possible via a clear path that builds upon what you've already learned.

Powerful -- Keras provides industry-strength performance and scalability: it is used by organizations and companies including NASA, YouTube, or Waymo.

## import data sets

using weather data from gsod
and gain data on ftp://ftp.ncdc.noaa.gov/pub/data/gsod

using data randomly choose from year 2021

In [3]:
def read_data(file_path):
    with open(file_path, 'r') as f:
        content = f.read()
    return content

## File walker

In [4]:
from os import walk

In [5]:
path = "data/GSOD_2021/"

In [6]:
for sdir, bdir, name in walk(path):
    file_name = name

In [7]:
random.choice(name)

'298690-99999-2021.op'

In [8]:
data = read_data(path+random.choice(name))

In [9]:
def org_file(data):
    """将温度信息抽取"""
    tem_avg, tem_max, tem_min = [], [], []
    for i in data[1:-1]:
        tem_avg.append(float(i[26:30]))
        tem_max.append(float(i[104:108]))
        tem_min.append(float(i[111:116]))
    return tem_avg

In [10]:
tem_avg = org_file(data.split("\n"))

In [11]:
def generate(train_set, data_set):
    """train_set: training set 的大小，
    data_set: 传入list, 数据本身"""
    test_set = len(data_set) - 8

    def get_data(set1, set2, data_set):
        train_data = []
        train_label = []
        for i in range(set1, set2):
            train_data.append(data_set[i:i + 7])
            train_label.append(data_set[i + 8])
        train_data = np.array(train_data)
        train_label = np.array(train_label)
        return train_data, train_label

    train_data, train_label = get_data(0, train_set, data_set)
    test_data, test_label = get_data(train_set, test_set, data_set)

    return (train_data, train_label), (test_data, test_label)

In [12]:
(train_data, train_label), (test_data, test_label) = generate(int(len(tem_avg)*0.75), tem_avg)

In [13]:
train_label, test_label = train_label.reshape(-1, 1), test_label.reshape(-1, 1)

In [14]:
test = train_data[0]

In [15]:
test = test.reshape(1,7)

In [16]:
def get_train_data(data):
    def polynize(dataset):
        dataset = dataset.reshape(1, -1)
        dataset_copy = dataset.copy()
        for i in range(2,3):
            dataset = np.append(dataset, dataset_copy**i, 1)
        return dataset
    def reshape(target_data, data):
        target_data = np.append((7,1), 0)
        target_data = data.reshape(-1)
        return target_data
        
    i =  0 
    out = np.zeros((data.shape[0], 14))
    for index in data:
        out[i] = reshape(data[i], polynize(index))
        i += 1
    return out

In [17]:
train_data_redo = get_train_data(train_data)
test_data_redo = get_train_data(test_data)

In [18]:
# train_data_redo.dtype = float, train_label.dtype = float
train_data_redo
train_label

train_data_redo, train_label = tf.convert_to_tensor(train_data_redo/100), tf.convert_to_tensor(train_label/100)

In [19]:
train_label

<tf.Tensor: shape=(150, 1), dtype=float64, numpy=
array([[0.404],
       [0.341],
       [0.396],
       [0.393],
       [0.391],
       [0.42 ],
       [0.533],
       [0.456],
       [0.415],
       [0.462],
       [0.446],
       [0.507],
       [0.556],
       [0.61 ],
       [0.594],
       [0.59 ],
       [0.67 ],
       [0.673],
       [0.636],
       [0.486],
       [0.423],
       [0.454],
       [0.631],
       [0.482],
       [0.413],
       [0.426],
       [0.456],
       [0.57 ],
       [0.467],
       [0.485],
       [0.481],
       [0.587],
       [0.587],
       [0.655],
       [0.541],
       [0.437],
       [0.412],
       [0.415],
       [0.293],
       [0.341],
       [0.438],
       [0.395],
       [0.412],
       [0.448],
       [0.604],
       [0.52 ],
       [0.498],
       [0.647],
       [0.645],
       [0.665],
       [0.714],
       [0.678],
       [0.521],
       [0.474],
       [0.513],
       [0.537],
       [0.58 ],
       [0.542],
       [0.517],
      

## tensorflow model

In [20]:
train_data_redo

<tf.Tensor: shape=(150, 14), dtype=float64, numpy=
array([[ 0.637 ,  0.546 ,  0.477 , ..., 27.7729, 21.9024, 29.0521],
       [ 0.546 ,  0.477 ,  0.44  , ..., 21.9024, 29.0521, 20.5209],
       [ 0.477 ,  0.44  ,  0.527 , ..., 29.0521, 20.5209, 16.3216],
       ...,
       [ 0.745 ,  0.766 ,  0.742 , ..., 57.9121, 57.3049, 61.1524],
       [ 0.766 ,  0.742 ,  0.74  , ..., 57.3049, 61.1524, 61.9369],
       [ 0.742 ,  0.74  ,  0.761 , ..., 61.1524, 61.9369, 62.0944]])>

In [21]:
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(14,)),
    tf.keras.layers.Dense(1)
])

In [22]:
model.compile(optimizer="adam",
             loss='mse')
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 14)                0         
_________________________________________________________________
dense (Dense)                (None, 1)                 15        
Total params: 15
Trainable params: 15
Non-trainable params: 0
_________________________________________________________________


In [23]:
model.fit(train_data_redo, train_label, batch_size=1, epochs=1000)

Epoch 1/1000
150/150 [==============================] - 0s 491us/step - loss: 78.1636
Epoch 2/1000
150/150 [==============================] - 0s 464us/step - loss: 62.4238
Epoch 3/1000
150/150 [==============================] - 0s 508us/step - loss: 55.5000
Epoch 4/1000
150/150 [==============================] - 0s 536us/step - loss: 49.5820
Epoch 5/1000
150/150 [==============================] - 0s 499us/step - loss: 44.1604

In [26]:
model.evaluate(test_data_redo, test_label)

2/2 [==============================] - 0s 3ms/step - loss: 1479.2220


1479.2220458984375

In [25]:
model.predict(train_data_redo[0])

ValueError: in user code:

    /Users/lishuyu/.conda/envs/nn_v0.1/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1569 predict_function  *
        return step_function(self, iterator)
    /Users/lishuyu/.conda/envs/nn_v0.1/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1559 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /Users/lishuyu/.conda/envs/nn_v0.1/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:1285 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /Users/lishuyu/.conda/envs/nn_v0.1/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2833 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /Users/lishuyu/.conda/envs/nn_v0.1/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:3608 _call_for_each_replica
        return fn(*args, **kwargs)
    

In [36]:
]

array([[-75.5],
       [-69.3],
       [-57.3],
       [-53.8],
       [-54.3],
       [-60.6],
       [-70.3],
       [-68.8],
       [-74.1],
       [-76.8],
       [-76.8],
       [-76.3],
       [-59.6],
       [-51.5],
       [-52.3],
       [-56.8],
       [-61.2],
       [-69.5],
       [-69.3],
       [-72.9],
       [-72.8],
       [-76.6],
       [-77.3],
       [-78.8],
       [-77.9],
       [-78.3],
       [-79.3],
       [-77.1],
       [-78.9],
       [-77. ],
       [-69.8],
       [-74. ],
       [-63.7],
       [-64.3],
       [-67.6],
       [-72.4],
       [-76.3],
       [-76.8]])